In [130]:
# Merge all
import glob
import json

json_files = glob.glob('articles/*.json')
print(json_files)

all_articles = []

for filepath in json_files:
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
        all_articles.extend(data)

with open('articles/240628_all_articles.json', 'w', encoding='utf-8') as file:
    json.dump(all_articles, file, ensure_ascii=False, indent=4)

['articles/NucNet_articles_NEW.json', 'articles/WNN_articles.json', 'articles/NCE_articles.json', 'articles/NEI_articles_NEW.json']


In [ ]:
import json
from datetime import datetime

# Check datetime format WNN
with open('articles/240627_all_articles.json', 'r') as file:
    data = json.load(file)

dates = set()

for article in data:
    date_str = article.get('date', None)  # Assuming 'date' is the key for date field
    if date_str:
        try:
            # Convert string to datetime object if necessary
            date_obj = datetime.strptime(date_str, '%d %B %Y')  # Adjust the format if necessary
            dates.add(date_obj.date())  # Store only the date part
        except ValueError:
            # Handle the case where the date format is not as expected
            print(f"Unexpected date format: {date_str}")

# Print all unique dates
for date in sorted(dates):
    print(date)

In [112]:
def count_words(paragraphs):
    if paragraphs is None:
        return 0
    return sum(len(paragraph.split()) for paragraph in paragraphs if paragraph)

def get_stats(data):
    num_articles_tota = f'{data.shape[0]:,}'
    num_words_total = sum(data['num_words'])

    rows = data.shape[0]
    cols = f'{data.shape[1]:,}'
    earliest = min(data['date']).date()
    latest = max(data['date']).date()

    article_breakdown = data.groupby('magasine').agg(
        num_articles=('text', 'size'),
        total_words=('num_words', 'sum')
    ).reset_index()
    article_breakdown['percentage_articles'] = (article_breakdown['num_articles'] / rows) * 100
    article_breakdown['percentage_words'] = (article_breakdown['total_words'] / num_words_total) * 100
    
    num_words_total = sum(data['num_words'])

    # Format numbers
    article_breakdown['num_articles'] = article_breakdown['num_articles'].apply(lambda x: f"{x:,}")
    article_breakdown['total_words'] = article_breakdown['total_words'].apply(lambda x: f"{x:,}")
    article_breakdown['percentage_articles'] = article_breakdown['percentage_articles'].apply(lambda x: f"{x:.2f}%")
    article_breakdown['percentage_words'] = article_breakdown['percentage_words'].apply(lambda x: f"{x:.2f}%")
    num_words_formatted = f'{num_words_total:,}'

    print(f'Total num articles: {num_articles_tota:>20}')
    print(f'Total num words: {num_words_formatted:>23}')
    print(f'Earliest date: {str(earliest):>25}')
    print(f'Latest date: {str(latest):>27}\n')
    print(article_breakdown)

def split_data(data, training_percent):
    length = data.shape[0]
    random_order = random.sample(range(length), length)
    train_order = random_order[:training_percent]

In [126]:
# Data exploration
import pandas as pd
import random
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

df = pd.read_json('articles/240627_all_articles.json')
#df['date'] = pd.to_datetime(df['date'])

In [127]:
df['num_words'] = df['text'].apply(count_words)

cutoff = '2021-06-14' # Exactly 2 years
df_new = df[df['date'] >= cutoff]
df_new.loc[:,'num_words'] = df_new['text'].apply(count_words)
get_stats(df_new)

AttributeError: 'str' object has no attribute 'date'